In [ ]:
import pandas as pd
import numpy as np
from itertools import permutations, product, combinations

In [ ]:
df_mod = pd.read_csv('mod.csv')
df_mod.columns
mod_mass_list = pd.unique(df_mod['Exact Mass'])
df_mod.rename(columns={'Exact Mass': 'Mass'}, inplace=True)

In [ ]:
def ppm2dm(m, ppm=10):
    return (m*ppm)/1E6

In [ ]:
def get_permutations(m, n):
    l = list(product(range(m+1), repeat=n))
    l = [i for i in l if sum(i) in range(m-3, m+2) and i[2] > 0]
    l = np.array(l)
    return l

In [ ]:
def handle_4bases(mass, masses=[329.0525, 305.0413, 345.0474, 306.0253]):
    size = int(mass/320)
    perms = get_permutations(size, len(masses))
    seq_masses = np.matmul(perms, masses)
    seq_masses_pd = pd.Series(seq_masses)
    res_masses = perms.copy()
    res_masses = pd.DataFrame(res_masses)
#     print(res_masses.index)
#     print(seq_masses_pd.index)
    res_masses['Mass'] = seq_masses_pd
    return res_masses[res_masses.Mass <= mass+1]
#     return seq_masses[seq_masses <= mass+1]

In [ ]:
def find_mass_in_df(mass, df, ppm=10):
    dm = ppm2dm(mass, ppm)
    low, high = mass - dm, mass + dm
#     print("low {} high {}".format(low, high))
    return df[(df.Mass>low) & (df.Mass<high)]
#     mask = np.ma.masked_inside(ds, low, high).mask
#     return ds[mask]

In [ ]:
dfm_rows = list(dfm.iterrows())
mass_cb = list()
mass_cb.extend(list(combinations(dfm_rows, 1)))
mass_cb.extend(list(combinations(dfm_rows, 2)))
mass_cb.extend(list(combinations(dfm_rows, 3)))

In [ ]:
mass_cb_sum = [(sum([item[1].Mass for item in row]), [item[0] for item in row]) for row in mass_cb]
mass_cb_sum.sort()
mass_cb_sum[:4]

In [ ]:
# mass = 2567.37756
mass = 4963.6122
# mass = 9183.4049
# mass = 13395.9393
seq_masses = handle_4bases(mass)
len(seq_masses)

diff = seq_masses
diff['Mass'] = mass - seq_masses['Mass']
# diff = pd.Series(diff)
# diff = diff[(diff.Mass > df_mod.Mass.min()) & (diff.Mass < df_mod.Mass.max())]
valid_diff = diff[diff.Mass > df_mod.Mass.min()]
# valid_diff.sort()
# valid_diff

count = 0
full_msg = ""
for idx, item in enumerate(mass_cb_sum):
    found = find_mass_in_df(item[0], valid_diff, 25)
    valid_found = found.dropna()
    if valid_found.size > 0:
#         print("found {} \n valid_found--->\n{}\n<---".format(mass_cb_sum[idx], valid_found.iloc[:, 0:4]))
        index = mass_cb_sum[idx][1]
#         print(list(dfm.loc[index]['Symbol']))
        
        for idx, row in valid_found.iterrows():
            msg = "{}A {}C {}G {}U ".format(int(row[0]), int(row[1]), int(row[2]), int(row[3])) 
            msg += "{}\n".format(' '.join(list(dfm.loc[index]['Symbol'])))
            full_msg += msg
            count += 1

print("found {} possibilities\n".format(count))
print(full_msg)